# Goal:

Create a Planner Agent, this planner agent creates a plan for a given task and then hands over it to another agent to execute the plan.

# Define LLM Connection

We define the LLM

In [14]:
from pydantic_settings import BaseSettings
import os
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("planner_agent")

class Settings(BaseSettings):
    GOOGLE_API_KEY: str
    MODEL_NAME: str
    CONTEXT7_API_KEY: str
    class Config:
        #ignore extra fields
        extra = "ignore"
        env_file = ".env"

settings = Settings()
os.environ["GOOGLE_API_KEY"] = settings.GOOGLE_API_KEY

# Define Agent

We define the Planer Agent with structured output and system prompt

In [2]:
from pydantic_ai import Agent
from pydantic import BaseModel, Field
  
class PlanOutput(BaseModel):
    outline: str = Field(description="The outline of the report")

planning_agent = Agent(
    model=settings.MODEL_NAME,
    system_prompt="""You are a planning assistant. Your Job is to create an outline for a report for an user. You have access to tools to help you answer questions.
1. Assess which tool you should use to answer the question.
2. Use get_human_in_the_loop to get user input in case of ambiguity. Provide your question.
3. Use create_plan to create a plan based on what you understand about what the user is asking for. The default outline is the following:
    - Summary 
    - Introduction
    - Sections and subsections
    - Conclusion
    - References
4. If you think the question is too complex or not relevant, first use get_human_in_the_loop to get user input. if user input is ambiguous, then respond with 'I don't know how to help you with that'.
5. After making the plan, use human_in_the_loop to ask the user if the plan looks good, if the user asks for changes, use_create_plan with the changes made by the user. If the user is happy with the plan, respond with the final plan.
""",
    output_type=PlanOutput
)

# Define Tools

We define 2 tools, one for planning and the other to include us in the loop

In [3]:
from pydantic import BaseModel
from pydantic_ai import RunContext

# Human in the loop tool
class GetHumanInTheLoopInput(BaseModel):
    """Input for getting more information"""
    questions: str

class GetHumanInTheLoopOutput(BaseModel):
    """Response for getting more information"""
    answer: str
    
@planning_agent.tool
def get_human_in_the_loop(_: RunContext[GetHumanInTheLoopInput], question: str) -> GetHumanInTheLoopOutput:
    input_str = input(f"Please provide your input for the question > '{question}': ")
    return GetHumanInTheLoopOutput(answer=input_str)

# Planning tool
class GetPlanInput(BaseModel):
    """Input for getting a plan"""
    instructions_for_plan: str

class GetPlanOutput(BaseModel):
    """Response for getting a plan"""
    plan: str

@planning_agent.tool
async def get_plan(_: RunContext[GetPlanInput], instructions_for_plan: str) -> GetPlanOutput:
    outline_agent = Agent(settings.MODEL_NAME,
                instructions="You are an experienced planner for a research project, you will be given a query from the user and you need to make an outline of the report to give the user about it, and what to search for in the web in order to create the report",
                output_type=GetPlanOutput)
    result = await outline_agent.run(instructions_for_plan)
    return GetPlanOutput(plan=result.output.plan)


# Run the Planner Agent

In [ ]:
user_request = "How to use pydantic-ai"
print(user_request)
# result = await planning_agent.run(user_request)

def helper_function_for_agent_run(agent: Agent, input_prompt: str):

    nodes = []
    async with planning_agent.iter(
        user_request,
    ) as agent_run:
        async for node in agent_run:
            # Each node represents a step in the agent's execution
            print(node)
            nodes.append(node)

How to use pydantic-ai
UserPromptNode(user_prompt='How to use pydantic-ai', instructions=None, instructions_functions=[], system_prompts=("You are a planning assistant. Your Job is to create an outline for a report for an user. You have access to tools to help you answer questions.\n1. Assess which tool you should use to answer the question.\n2. Use get_human_in_the_loop to get user input in case of ambiguity. Provide your question.\n3. Use create_plan to create a plan based on what you understand about what the user is asking for. The default outline is the following:\n    - Summary \n    - Introduction\n    - Sections and subsections\n    - Conclusion\n    - References\n4. If you think the question is too complex or not relevant, first use get_human_in_the_loop to get user input. if user input is ambiguous, then respond with 'I don't know how to help you with that'.\n5. After making the plan, use human_in_the_loop to ask the user if the plan looks good, if the user asks for changes, 

# Display Results

In [26]:
print(nodes[-1].data.output.outline)

Report Outline: How to Use Pydantic-AI for AI Agents

1.  **Introduction to Pydantic-AI for AI Agents**
    *   Brief overview of Pydantic and its benefits.
    *   Introduction to `pydantic-ai` and its relevance in AI agent development.
    *   Why structured data and validation are crucial for reliable AI agents.

2.  **Setting Up and Core Concepts**
    *   Installation of `pydantic-ai`.
    *   Defining structured data for agent inputs, outputs, and internal states using Pydantic models.
    *   Implementing input and output validation for agent interactions.

3.  **Pydantic-AI in Agent Development: Practical Examples**
    *   **Defining Agent Tools:** Using Pydantic to describe tool interfaces (parameters, return types) for function calling.
    *   **Validating Agent Reasoning and Actions:** Ensuring agent-generated plans or actions conform to predefined structures.
    *   **Handling Agent Observations and State:** Structuring and validating data received from the environment o

## Conclusion

In this notebook, we demonstrated how to create a sophisticated **Planner Agent** that can create structured plans for complex tasks:

### Key Takeaways

1. **Agent Coordination**: Built a planner agent that creates plans and hands them off to other agents for execution
2. **Human-in-the-Loop**: Implemented interactive tools that allow the agent to ask clarifying questions when faced with ambiguous requests
3. **Nested Agent Architecture**: Created a planning tool that internally uses another specialized agent for outline generation
4. **Structured Planning Output**: Used Pydantic models to ensure plans are returned in a consistent, well-defined format
5. **Interactive Planning Process**: Demonstrated how agents can engage users to refine requirements before creating detailed plans

### What We Learned

- **Agent Specialization**: Different agents can be designed for specific tasks (planning vs. execution)
- **Interactive Decision Making**: Agents can pause execution to gather more information from users
- **Hierarchical Agent Systems**: Agents can create and coordinate with other agents to accomplish complex tasks
- **Structured Communication**: Pydantic models ensure reliable data exchange between agents and users
- **Adaptive Planning**: The agent can adjust its planning approach based on user input and clarification

### The Planning Process Demonstrated

1. **Assessment**: Agent evaluates the complexity and clarity of the user request
2. **Clarification**: Uses human-in-the-loop tool to resolve ambiguities
3. **Plan Generation**: Creates a detailed, structured outline using a specialized planning sub-agent
4. **Structured Output**: Returns the plan in a well-defined format ready for execution

This planner agent pattern is essential for building reliable AI systems that can handle complex, multi-step tasks while maintaining user control and transparency in the planning process.
